In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
df = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv")
EXCLUDED_COLS = {'date_id', 'forward_returns', 'risk_free_rate', 'market_forward_excess_returns', 'log_market_forward_excess_returns'}
TARGET_COL = "log_market_forward_excess_returns"
DATE_COL = "date_id"
SENTINAL_VALUE = -9999


def sort_data_by_date_id(df, date_col):
    return df.sort_values(date_col).reset_index(drop=True)

def create_target_log(df, target_col):
    log_target_col = np.log1p(df[target_col])
    df_out = pd.concat([df, log_target_col], axis=1)
    return df_out

def create_category_groups(df, excluded_col):
    category_groups = {
        col[0]: []
        for col in df.columns
        if col[0].isalpha() and col not in excluded_col
    }
    for dfcol in df.columns:
        if dfcol in excluded_col:
            continue
        if dfcol and dfcol[0].isalpha() and dfcol[0] in category_groups:
            category = dfcol[0]
            category_groups[category].append(dfcol)

    return category_groups

def construct_momentum_cols(
        df, target, sentinal, category_groups,
        windows=[5, 10, 21, 63, 126, 252],
        lags=[1, 2, 3, 5, 10, 21, 63, 126]
        ):
    df_momentum = pd.DataFrame()

    target_shifted = df[target].shift(1)

    for window in windows:
        df_momentum[f'MOM_mean_{window}'] = target_shifted.rolling(window).mean()
        df_momentum[f'MOM_std_{window}'] = target_shifted.rolling(window).std()
        df_momentum[f'MOM_roc_{window}'] = target_shifted / target_shifted.shift(window) - 1
        df_momentum[f'MOM_cum_{window}'] = target_shifted.rolling(window).sum()

    df_momentum['MOM_ema_fast'] = target_shifted.ewm(span=10, adjust=False).mean()
    df_momentum['MOM_ema_slow'] = target_shifted.ewm(span=50, adjust=False).mean()
    df_momentum['MOM_ema_diff'] = df_momentum['MOM_ema_fast'] - df_momentum['MOM_ema_slow']
    df_momentum['MOM_ema_ratio'] = df_momentum['MOM_ema_fast'] / df_momentum['MOM_ema_slow'] - 1

    df_momentum['MOM_std_21_252_ratio'] = df_momentum['MOM_std_21'] / df_momentum['MOM_std_252']

    for lag in lags:
        df_momentum[f'MOM_lag_{lag}'] = df[target].shift(lag)

    df_momentum = df_momentum.fillna(sentinal)

    mom_cols = df_momentum.columns()

    if 'MOM' not in category_groups:
        category_groups["MOM"] = []
    category_groups_out = category_groups["MOM"].extend(mom_cols)

    df_out = pd.concat([df, df_momentum], axis=1)

    return df_out, category_groups_out

def construct_volatility_indicators(
            df, target, sentinal, category_groups
        ):
    df_volatility_indicators = pd.DataFrame()

    target_shifted = df[target].shift(1)

    df_volatility_indicators["regime_highvol"] = (df["vol_ratio"] > 1.2).astype("int8")

    z = target_shifted / 


    return df_out
    

df = sort_data_by_date_id(df, DATE_COL)
df = create_target_log(df, TARGET_COL)
category_groups = create_category_groups(df, EXCLUDED_COLS)
df, category_groups = construct_momentum_cols(df, target=TARGET_COL, sentinal=SENTINAL_VALUE, category_groups=category_groups)




FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/hull-tactical-market-prediction/train.csv'